# Entraînement des modèles pour la détection de pneumonie (Suite)

Ce notebook est la suite du notebook `02_entrainement_modeles.ipynb` et se concentre sur l'entraînement et l'évaluation des modèles.

In [ ]:
# Importation des bibliothèques nécessaires
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, precision_recall_curve

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model, save_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Input, BatchNormalization
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Pour éviter les avertissements
import warnings
warnings.filterwarnings('ignore')

# Configuration de l'affichage
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (12, 8)

## 4. Entraînement des modèles

Nous allons maintenant entraîner les différents modèles que nous avons créés.

In [ ]:
# Charger les données préparées dans le notebook précédent
# Si vous exécutez ce notebook séparément, vous devrez d'abord exécuter le code de chargement des données
# du notebook précédent ou charger les données à partir d'un fichier sauvegardé

# Fonction pour entraîner un modèle
def train_model(model, X_train, y_train, X_val, y_val, model_name='model', 
                batch_size=32, epochs=20, use_augmentation=True):
    """
    Entraîne un modèle avec ou sans augmentation de données.
    
    Args:
        model: Modèle à entraîner
        X_train (numpy.ndarray): Images d'entraînement
        y_train (numpy.ndarray): Étiquettes d'entraînement
        X_val (numpy.ndarray): Images de validation
        y_val (numpy.ndarray): Étiquettes de validation
        model_name (str): Nom du modèle pour la sauvegarde
        batch_size (int): Taille du batch
        epochs (int): Nombre d'époques
        use_augmentation (bool): Utiliser l'augmentation de données
        
    Returns:
        tuple: Modèle entraîné et historique d'entraînement
    """
    # Vérifier si les données d'entraînement sont disponibles
    if len(X_train) == 0 or len(y_train) == 0:
        print("Erreur: Données d'entraînement vides.")
        return None, None
    
    # Créer le répertoire de résultats s'il n'existe pas
    results_dir = r'C:\Users\Krikri\Documents\EPITECH\MSc\Msc1\T-DEV-810-PAR-29\epitech_docs\detection_pneumonie_ia\results'
    os.makedirs(results_dir, exist_ok=True)
    
    # Définir les callbacks
    callbacks = [
        # Sauvegarde du meilleur modèle
        ModelCheckpoint(
            filepath=os.path.join(results_dir, f'{model_name}.h5'),
            monitor='val_accuracy',
            mode='max',
            save_best_only=True,
            verbose=1
        ),
        # Arrêt anticipé si pas d'amélioration
        EarlyStopping(
            monitor='val_loss',
            patience=5,
            verbose=1,
            restore_best_weights=True  # Restaurer les meilleurs poids
        ),
        # Réduction du taux d'apprentissage si plateau
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=3,
            verbose=1,
            min_lr=1e-6
        )
    ]
    
    print(f"Début de l'entraînement du modèle {model_name}...")
    print(f"Nombre d'images d'entraînement: {len(X_train)}")
    print(f"Nombre d'images de validation: {len(X_val)}")
    print(f"Utilisation de l'augmentation de données: {use_augmentation}")
    
    try:
        # Entraînement avec ou sans augmentation de données
        if use_augmentation:
            # Configurer le générateur d'augmentation
            datagen = ImageDataGenerator(
                rotation_range=15,
                width_shift_range=0.1,
                height_shift_range=0.1,
                shear_range=0.1,
                zoom_range=0.1,
                horizontal_flip=True,
                fill_mode='nearest'
            )
            
            # Adapter le générateur aux données d'entraînement
            datagen.fit(X_train)
            
            # Calculer le nombre d'étapes par époque
            steps_per_epoch = len(X_train) // batch_size
            if steps_per_epoch == 0:  # Éviter les divisions par zéro
                steps_per_epoch = 1
            
            # Entraîner le modèle avec le générateur
            history = model.fit(
                datagen.flow(X_train, y_train, batch_size=batch_size),
                steps_per_epoch=steps_per_epoch,
                epochs=epochs,
                validation_data=(X_val, y_val),
                callbacks=callbacks,
                verbose=1
            )
        else:
            # Entraîner le modèle sans augmentation
            history = model.fit(
                X_train, y_train,
                batch_size=batch_size,
                epochs=epochs,
                validation_data=(X_val, y_val),
                callbacks=callbacks,
                verbose=1
            )
        
        print(f"Entraînement terminé après {len(history.epoch)} époques.")
        
        # Vérifier si le fichier du modèle existe
        model_path = os.path.join(results_dir, f'{model_name}.h5')
        if os.path.exists(model_path):
            # Charger le meilleur modèle
            print(f"Chargement du meilleur modèle depuis {model_path}")
            best_model = load_model(model_path)
            return best_model, history
        else:
            print(f"Attention: Le fichier du modèle {model_path} n'a pas été créé.")
            return model, history  # Retourner le modèle actuel
            
    except Exception as e:
        print(f"Erreur lors de l'entraînement du modèle {model_name}: {e}")
        return model, None  # Retourner le modèle non entraîné en cas d'erreur

# Fonction pour visualiser l'historique d'entraînement
def plot_training_history(history, model_name='Modèle'):
    """
    Visualise l'historique d'entraînement d'un modèle.
    
    Args:
        history: Historique d'entraînement
        model_name (str): Nom du modèle
    """
    # Vérifier si l'historique est valide
    if history is None or not hasattr(history, 'history') or not history.history:
        print(f"Aucun historique d'entraînement valide disponible pour {model_name}.")
        return
    
    # Créer une figure avec deux sous-graphiques
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Graphique de l'accuracy
    ax1.plot(history.history['accuracy'], label='Entraînement')
    ax1.plot(history.history['val_accuracy'], label='Validation')
    ax1.set_title(f'Accuracy - {model_name}')
    ax1.set_xlabel('Époque')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Graphique de la loss
    ax2.plot(history.history['loss'], label='Entraînement')
    ax2.plot(history.history['val_loss'], label='Validation')
    ax2.set_title(f'Loss - {model_name}')
    ax2.set_xlabel('Époque')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Afficher des informations supplémentaires sur la convergence
    epochs = len(history.history['accuracy'])
    final_train_acc = history.history['accuracy'][-1]
    final_val_acc = history.history['val_accuracy'][-1]
    final_train_loss = history.history['loss'][-1]
    final_val_loss = history.history['val_loss'][-1]
    
    print(f"\nRésumé de l'entraînement pour {model_name}:")
    print(f"Nombre total d'époques: {epochs}")
    print(f"Accuracy finale - Entraînement: {final_train_acc:.4f}, Validation: {final_val_acc:.4f}")
    print(f"Loss finale - Entraînement: {final_train_loss:.4f}, Validation: {final_val_loss:.4f}")
    
    # Vérifier si l'entraînement a convergé
    if epochs > 5:
        last_val_losses = history.history['val_loss'][-5:]
        loss_change = (last_val_losses[-1] - last_val_losses[0]) / last_val_losses[0]
        
        if abs(loss_change) < 0.01:
            print("L'entraînement a convergé (variation de loss < 1% sur les 5 dernières époques).")
        elif loss_change > 0:
            print("Attention: La loss de validation augmente, ce qui peut indiquer un surapprentissage.")
        else:
            print("L'entraînement pourrait bénéficier de plus d'époques (la loss continue de diminuer).")
    
    # Vérifier l'écart entre l'entraînement et la validation
    train_val_acc_gap = final_train_acc - final_val_acc
    if train_val_acc_gap > 0.1:
        print(f"Attention: Écart important entre l'accuracy d'entraînement et de validation ({train_val_acc_gap:.4f}), possible surapprentissage.")

# Fonction pour vérifier l'état d'avancement de l'entraînement
def check_training_status(model_name='model'):
    """
    Vérifie si un modèle a été entraîné et sauvegardé.
    
    Args:
        model_name (str): Nom du modèle
        
    Returns:
        bool: True si le modèle est entraîné, False sinon
    """
    results_dir = r'C:\Users\Krikri\Documents\EPITECH\MSc\Msc1\T-DEV-810-PAR-29\epitech_docs\detection_pneumonie_ia\results'
    model_path = os.path.join(results_dir, f'{model_name}.h5')
    
    if os.path.exists(model_path):
        model_size = os.path.getsize(model_path) / (1024 * 1024)  # Taille en MB
        print(f"Le modèle {model_name} est entraîné et sauvegardé ({model_size:.2f} MB).")
        return True
    else:
        print(f"Le modèle {model_name} n'est pas encore entraîné ou n'a pas été sauvegardé.")
        return False

### 4.1 Entraînement du CNN simple

In [ ]:
# Vérifier si les données sont disponibles
if len(X_train) == 0 or len(X_val) == 0:
    print("Erreur: Les données d'entraînement ou de validation sont vides.")
    print("Veuillez d'abord exécuter le code de chargement des données.")
else:
    try:
        # Créer le modèle CNN
        print("Création du modèle CNN...")
        cnn_model = create_cnn_model(input_shape=(224, 224, 3))
        
        # Afficher le résumé du modèle
        print("\nRésumé du modèle CNN:")
        cnn_model.summary()
        
        # Vérifier si le modèle est déjà entraîné
        results_dir = r'C:\Users\Krikri\Documents\EPITECH\MSc\Msc1\T-DEV-810-PAR-29\epitech_docs\detection_pneumonie_ia\results'
        model_path = os.path.join(results_dir, 'cnn_model.h5')
        
        if os.path.exists(model_path):
            print(f"\nLe modèle CNN est déjà entraîné. Voulez-vous le charger? (y/n)")
            # Dans un notebook interactif, vous pourriez utiliser input() ici
            # Comme c'est un script, nous supposons que nous voulons réentraîner
            load_existing = False
            
            if load_existing:
                print("Chargement du modèle CNN existant...")
                cnn_model = load_model(model_path)
                print("Modèle chargé avec succès.")
                
                # Pour visualiser l'historique, nous aurions besoin de le charger séparément
                # car il n'est pas sauvegardé avec le modèle
                print("Note: L'historique d'entraînement n'est pas disponible pour un modèle chargé.")
            else:
                # Entraîner le modèle
                print("\nRéentraînement du modèle CNN...")
                cnn_model, cnn_history = train_model(
                    cnn_model, X_train, y_train, X_val, y_val,
                    model_name='cnn_model',
                    batch_size=32,
                    epochs=20,
                    use_augmentation=True
                )
                
                # Visualiser l'historique d'entraînement
                if cnn_history is not None:
                    plot_training_history(cnn_history, model_name='CNN')
                else:
                    print("Erreur: L'entraînement n'a pas produit d'historique valide.")
        else:
            # Entraîner le modèle
            print("\nEntraînement du modèle CNN...")
            cnn_model, cnn_history = train_model(
                cnn_model, X_train, y_train, X_val, y_val,
                model_name='cnn_model',
                batch_size=32,
                epochs=20,
                use_augmentation=True
            )
            
            # Visualiser l'historique d'entraînement
            if cnn_history is not None:
                plot_training_history(cnn_history, model_name='CNN')
            else:
                print("Erreur: L'entraînement n'a pas produit d'historique valide.")
        
        # Vérifier si le modèle est bien entraîné
        if cnn_model is not None:
            print("\nÉvaluation du modèle CNN sur l'ensemble de validation...")
            val_loss, val_acc = cnn_model.evaluate(X_val, y_val, verbose=1)
            print(f"Validation - Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}")
            
            # Faire des prédictions sur quelques exemples
            if len(X_val) > 0:
                print("\nPrédictions sur quelques exemples de validation:")
                num_examples = min(5, len(X_val))
                pred_indices = np.random.choice(len(X_val), num_examples, replace=False)
                
                for i, idx in enumerate(pred_indices):
                    img = X_val[idx]
                    true_label = y_val[idx]
                    pred_prob = cnn_model.predict(np.expand_dims(img, axis=0))[0][0]
                    pred_label = 1 if pred_prob > 0.5 else 0
                    
                    print(f"Exemple {i+1}: Vraie classe = {'Pneumonie' if true_label == 1 else 'Normal'}, "
                          f"Prédiction = {'Pneumonie' if pred_label == 1 else 'Normal'} "
                          f"(confiance: {pred_prob:.2f})")
        else:
            print("Erreur: Le modèle CNN n'a pas été correctement créé ou entraîné.")
    
    except Exception as e:
        print(f"Une erreur s'est produite lors de la création ou de l'entraînement du modèle CNN: {e}")

### 4.2 Entraînement du modèle VGG16

In [ ]:
# Vérifier si les données sont disponibles
if len(X_train) == 0 or len(X_val) == 0:
    print("Erreur: Les données d'entraînement ou de validation sont vides.")
    print("Veuillez d'abord exécuter le code de chargement des données.")
else:
    try:
        # Vérifier si le modèle est déjà entraîné
        results_dir = r'C:\Users\Krikri\Documents\EPITECH\MSc\Msc1\T-DEV-810-PAR-29\epitech_docs\detection_pneumonie_ia\results'
        os.makedirs(results_dir, exist_ok=True)
        model_path = os.path.join(results_dir, 'vgg16_model.h5')
        
        if os.path.exists(model_path):
            print(f"\nLe modèle VGG16 est déjà entraîné. Voulez-vous le charger? (y/n)")
            # Dans un notebook interactif, vous pourriez utiliser input() ici
            # Comme c'est un script, nous supposons que nous voulons réentraîner
            load_existing = False
            
            if load_existing:
                print("Chargement du modèle VGG16 existant...")
                vgg16_model = load_model(model_path)
                print("Modèle chargé avec succès.")
                
                # Pour visualiser l'historique, nous aurions besoin de le charger séparément
                print("Note: L'historique d'entraînement n'est pas disponible pour un modèle chargé.")
            else:
                # Créer le modèle VGG16
                print("Création du modèle VGG16...")
                vgg16_model = create_vgg16_model(input_shape=(224, 224, 3))
                
                # Afficher un résumé simplifié du modèle (VGG16 a beaucoup de couches)
                print("\nRésumé du modèle VGG16:")
                print(f"Nombre total de couches: {len(vgg16_model.layers)}")
                print(f"Nombre de paramètres: {vgg16_model.count_params():,}")
                print("Architecture: VGG16 (base) + Flatten + Dense(512) + BatchNorm + Dropout(0.5) + Dense(1, sigmoid)")
                
                # Entraîner le modèle
                print("\nRéentraînement du modèle VGG16...")
                # Libérer la mémoire avant l'entraînement
                import gc
                gc.collect()
                tf.keras.backend.clear_session()
                
                vgg16_model, vgg16_history = train_model(
                    vgg16_model, X_train, y_train, X_val, y_val,
                    model_name='vgg16_model',
                    batch_size=16,  # Batch size plus petit pour VGG16 (modèle plus grand)
                    epochs=15,
                    use_augmentation=True
                )
                
                # Visualiser l'historique d'entraînement
                if vgg16_history is not None:
                    plot_training_history(vgg16_history, model_name='VGG16')
                else:
                    print("Erreur: L'entraînement n'a pas produit d'historique valide.")
        else:
            # Créer le modèle VGG16
            print("Création du modèle VGG16...")
            vgg16_model = create_vgg16_model(input_shape=(224, 224, 3))
            
            # Afficher un résumé simplifié du modèle (VGG16 a beaucoup de couches)
            print("\nRésumé du modèle VGG16:")
            print(f"Nombre total de couches: {len(vgg16_model.layers)}")
            print(f"Nombre de paramètres: {vgg16_model.count_params():,}")
            print("Architecture: VGG16 (base) + Flatten + Dense(512) + BatchNorm + Dropout(0.5) + Dense(1, sigmoid)")
            
            # Entraîner le modèle
            print("\nEntraînement du modèle VGG16...")
            # Libérer la mémoire avant l'entraînement
            import gc
            gc.collect()
            tf.keras.backend.clear_session()
            
            vgg16_model, vgg16_history = train_model(
                vgg16_model, X_train, y_train, X_val, y_val,
                model_name='vgg16_model',
                batch_size=16,  # Batch size plus petit pour VGG16 (modèle plus grand)
                epochs=15,
                use_augmentation=True
            )
            
            # Visualiser l'historique d'entraînement
            if vgg16_history is not None:
                plot_training_history(vgg16_history, model_name='VGG16')
            else:
                print("Erreur: L'entraînement n'a pas produit d'historique valide.")
        
        # Vérifier si le modèle est bien entraîné
        if vgg16_model is not None:
            print("\nÉvaluation du modèle VGG16 sur l'ensemble de validation...")
            val_loss, val_acc = vgg16_model.evaluate(X_val, y_val, verbose=1)
            print(f"Validation - Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}")
            
            # Faire des prédictions sur quelques exemples
            if len(X_val) > 0:
                print("\nPrédictions sur quelques exemples de validation:")
                num_examples = min(5, len(X_val))
                pred_indices = np.random.choice(len(X_val), num_examples, replace=False)
                
                for i, idx in enumerate(pred_indices):
                    img = X_val[idx]
                    true_label = y_val[idx]
                    pred_prob = vgg16_model.predict(np.expand_dims(img, axis=0))[0][0]
                    pred_label = 1 if pred_prob > 0.5 else 0
                    
                    print(f"Exemple {i+1}: Vraie classe = {'Pneumonie' if true_label == 1 else 'Normal'}, "
                          f"Prédiction = {'Pneumonie' if pred_label == 1 else 'Normal'} "
                          f"(confiance: {pred_prob:.2f})")
        else:
            print("Erreur: Le modèle VGG16 n'a pas été correctement créé ou entraîné.")
    
    except Exception as e:
        print(f"Une erreur s'est produite lors de la création ou de l'entraînement du modèle VGG16: {e}")
        import traceback
        traceback.print_exc()

### 4.3 Entraînement du modèle ResNet50

In [ ]:
# Vérifier si les données sont disponibles
if len(X_train) == 0 or len(X_val) == 0:
    print("Erreur: Les données d'entraînement ou de validation sont vides.")
    print("Veuillez d'abord exécuter le code de chargement des données.")
else:
    try:
        # Vérifier si le modèle est déjà entraîné
        results_dir = r'C:\Users\Krikri\Documents\EPITECH\MSc\Msc1\T-DEV-810-PAR-29\epitech_docs\detection_pneumonie_ia\results'
        os.makedirs(results_dir, exist_ok=True)
        model_path = os.path.join(results_dir, 'resnet50_model.h5')
        
        if os.path.exists(model_path):
            print(f"\nLe modèle ResNet50 est déjà entraîné. Voulez-vous le charger? (y/n)")
            # Dans un notebook interactif, vous pourriez utiliser input() ici
            # Comme c'est un script, nous supposons que nous voulons réentraîner
            load_existing = False
            
            if load_existing:
                print("Chargement du modèle ResNet50 existant...")
                resnet50_model = load_model(model_path)
                print("Modèle chargé avec succès.")
                
                # Pour visualiser l'historique, nous aurions besoin de le charger séparément
                print("Note: L'historique d'entraînement n'est pas disponible pour un modèle chargé.")
            else:
                # Libérer la mémoire avant de créer un nouveau modèle
                import gc
                gc.collect()
                tf.keras.backend.clear_session()
                
                # Créer le modèle ResNet50
                print("Création du modèle ResNet50...")
                resnet50_model = create_resnet50_model(input_shape=(224, 224, 3))
                
                # Afficher un résumé simplifié du modèle (ResNet50 a beaucoup de couches)
                print("\nRésumé du modèle ResNet50:")
                print(f"Nombre total de couches: {len(resnet50_model.layers)}")
                print(f"Nombre de paramètres: {resnet50_model.count_params():,}")
                print("Architecture: ResNet50 (base) + Flatten + Dense(512) + BatchNorm + Dropout(0.5) + Dense(1, sigmoid)")
                
                # Entraîner le modèle
                print("\nRéentraînement du modèle ResNet50...")
                # Libérer la mémoire avant l'entraînement
                gc.collect()
                
                # Réduire la taille du batch si nécessaire pour éviter les problèmes de mémoire
                batch_size = 8  # Encore plus petit pour ResNet50 qui est très volumineux
                
                resnet50_model, resnet50_history = train_model(
                    resnet50_model, X_train, y_train, X_val, y_val,
                    model_name='resnet50_model',
                    batch_size=batch_size,
                    epochs=15,
                    use_augmentation=True
                )
                
                # Visualiser l'historique d'entraînement
                if resnet50_history is not None:
                    plot_training_history(resnet50_history, model_name='ResNet50')
                else:
                    print("Erreur: L'entraînement n'a pas produit d'historique valide.")
        else:
            # Libérer la mémoire avant de créer un nouveau modèle
            import gc
            gc.collect()
            tf.keras.backend.clear_session()
            
            # Créer le modèle ResNet50
            print("Création du modèle ResNet50...")
            resnet50_model = create_resnet50_model(input_shape=(224, 224, 3))
            
            # Afficher un résumé simplifié du modèle (ResNet50 a beaucoup de couches)
            print("\nRésumé du modèle ResNet50:")
            print(f"Nombre total de couches: {len(resnet50_model.layers)}")
            print(f"Nombre de paramètres: {resnet50_model.count_params():,}")
            print("Architecture: ResNet50 (base) + Flatten + Dense(512) + BatchNorm + Dropout(0.5) + Dense(1, sigmoid)")
            
            # Entraîner le modèle
            print("\nEntraînement du modèle ResNet50...")
            # Libérer la mémoire avant l'entraînement
            gc.collect()
            
            # Réduire la taille du batch si nécessaire pour éviter les problèmes de mémoire
            batch_size = 8  # Encore plus petit pour ResNet50 qui est très volumineux
            
            resnet50_model, resnet50_history = train_model(
                resnet50_model, X_train, y_train, X_val, y_val,
                model_name='resnet50_model',
                batch_size=batch_size,
                epochs=15,
                use_augmentation=True
            )
            
            # Visualiser l'historique d'entraînement
            if resnet50_history is not None:
                plot_training_history(resnet50_history, model_name='ResNet50')
            else:
                print("Erreur: L'entraînement n'a pas produit d'historique valide.")
        
        # Vérifier si le modèle est bien entraîné
        if resnet50_model is not None:
            print("\nÉvaluation du modèle ResNet50 sur l'ensemble de validation...")
            # Utiliser un batch_size plus petit pour l'évaluation aussi
            val_loss, val_acc = resnet50_model.evaluate(X_val, y_val, batch_size=8, verbose=1)
            print(f"Validation - Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}")
            
            # Faire des prédictions sur quelques exemples
            if len(X_val) > 0:
                print("\nPrédictions sur quelques exemples de validation:")
                num_examples = min(5, len(X_val))
                pred_indices = np.random.choice(len(X_val), num_examples, replace=False)
                
                for i, idx in enumerate(pred_indices):
                    img = X_val[idx]
                    true_label = y_val[idx]
                    pred_prob = resnet50_model.predict(np.expand_dims(img, axis=0), batch_size=1)[0][0]
                    pred_label = 1 if pred_prob > 0.5 else 0
                    
                    print(f"Exemple {i+1}: Vraie classe = {'Pneumonie' if true_label == 1 else 'Normal'}, "
                          f"Prédiction = {'Pneumonie' if pred_label == 1 else 'Normal'} "
                          f"(confiance: {pred_prob:.2f})")
        else:
            print("Erreur: Le modèle ResNet50 n'a pas été correctement créé ou entraîné.")
    
    except Exception as e:
        print(f"Une erreur s'est produite lors de la création ou de l'entraînement du modèle ResNet50: {e}")
        import traceback
        traceback.print_exc()
        
        print("\nConseil: ResNet50 est un modèle très volumineux qui peut nécessiter beaucoup de mémoire.")
        print("Si vous rencontrez des problèmes de mémoire, essayez de:")
        print("1. Réduire la taille du batch (par exemple, batch_size=4)")
        print("2. Réduire la taille des images d'entrée (par exemple, input_shape=(150, 150, 3))")
        print("3. Utiliser moins d'images pour l'entraînement")
        print("4. Utiliser un modèle plus léger comme le CNN simple ou MobileNet")

## 5. Évaluation des modèles sur l'ensemble de test

Nous allons maintenant évaluer les performances des modèles entraînés sur l'ensemble de test.

In [ ]:
def evaluate_model(model, X_test, y_test, model_name='Modèle', class_names=['Normal', 'Pneumonie']):
    """
    Évalue les performances d'un modèle sur l'ensemble de test.
    
    Args:
        model: Modèle à évaluer
        X_test (numpy.ndarray): Images de test
        y_test (numpy.ndarray): Étiquettes de test
        model_name (str): Nom du modèle
        class_names (list): Noms des classes
        
    Returns:
        dict: Métriques d'évaluation
    """
    # Vérifier si le modèle est entraîné
    if model is None:
        print(f"Le modèle {model_name} n'est pas disponible.")
        return None
    
    # Vérifier si les données de test sont disponibles
    if len(X_test) == 0 or len(y_test) == 0:
        print(f"Les données de test sont vides. Impossible d'évaluer le modèle {model_name}.")
        return None
    
    try:
        # Faire des prédictions
        print(f"Prédiction sur {len(X_test)} images de test...")
        # Utiliser un batch_size plus petit pour les modèles volumineux
        batch_size = 16
        if 'resnet' in model_name.lower() or 'vgg' in model_name.lower():
            batch_size = 8
        
        y_pred_proba = model.predict(X_test, batch_size=batch_size, verbose=1)
        
        # S'assurer que y_pred_proba est un tableau 1D
        if len(y_pred_proba.shape) > 1 and y_pred_proba.shape[1] > 1:
            # Pour les modèles avec plusieurs sorties (softmax)
            y_pred_proba = y_pred_proba[:, 1]
        else:
            # Pour les modèles avec une seule sortie (sigmoid)
            y_pred_proba = y_pred_proba.flatten()
            
        y_pred = (y_pred_proba >= 0.5).astype(int)
        
        # Calculer les métriques
        metrics = {
            'accuracy': accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred),
            'recall': recall_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred),
            'auc': roc_auc_score(y_test, y_pred_proba)
        }
        
        # Afficher les résultats
        print(f"\n=== Résultats d'évaluation pour {model_name} ===")
        print("=" * 50)
        print(f"Accuracy: {metrics['accuracy']:.4f}")
        print(f"Precision: {metrics['precision']:.4f}")
        print(f"Recall: {metrics['recall']:.4f}")
        print(f"F1-score: {metrics['f1_score']:.4f}")
        print(f"AUC: {metrics['auc']:.4f}")
        
        # Matrice de confusion
        cm = confusion_matrix(y_test, y_pred)
        print("\nMatrice de confusion:")
        print(cm)
        
        # Vérifier si c'est bien une matrice 2x2 avant de la décomposer
        if cm.shape == (2, 2):
            # Calculer les valeurs pour la matrice de confusion
            tn, fp, fn, tp = cm.ravel()
            total = tn + fp + fn + tp
            
            print(f"\nDétails de la matrice de confusion:")
            print(f"Vrais Négatifs (Normal correctement identifié): {tn} ({tn/total*100:.1f}%)")
            print(f"Faux Positifs (Normal prédit comme Pneumonie): {fp} ({fp/total*100:.1f}%)")
            print(f"Faux Négatifs (Pneumonie prédite comme Normal): {fn} ({fn/total*100:.1f}%)")
            print(f"Vrais Positifs (Pneumonie correctement identifiée): {tp} ({tp/total*100:.1f}%)")
            
            # Calculer des métriques supplémentaires
            specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
            npv = tn / (tn + fn) if (tn + fn) > 0 else 0  # Negative Predictive Value
            
            print(f"\nMétriques supplémentaires:")
            print(f"Spécificité: {specificity:.4f}")
            print(f"Valeur prédictive négative: {npv:.4f}")
        else:
            print("Note: La matrice de confusion n'est pas de taille 2x2, certaines métriques détaillées ne sont pas calculées.")
        
        # Rapport de classification
        print("\nRapport de classification:")
        print(classification_report(y_test, y_pred, target_names=class_names))
        
        # Visualiser la matrice de confusion
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=class_names, 
                    yticklabels=class_names)
        plt.xlabel('Prédiction')
        plt.ylabel('Vérité terrain')
        plt.title(f'Matrice de confusion - {model_name}')
        
        # Ajouter les pourcentages si c'est une matrice 2x2
        if cm.shape == (2, 2):
            for i in range(len(class_names)):
                for j in range(len(class_names)):
                    plt.text(j + 0.5, i + 0.7, f'({cm[i, j]/np.sum(cm)*100:.1f}%)', 
                             ha='center', va='center', color='black' if cm[i, j] < cm.max()/2 else 'white')
        
        plt.tight_layout()
        plt.show()
        
        # Courbe ROC
        fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
        plt.figure(figsize=(10, 8))
        plt.plot(fpr, tpr, 'b-', linewidth=2, label=f'AUC = {metrics["auc"]:.4f}')
        plt.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Aléatoire (AUC = 0.5)')
        plt.xlabel('Taux de faux positifs (1 - Spécificité)')
        plt.ylabel('Taux de vrais positifs (Sensibilité)')
        plt.title(f'Courbe ROC - {model_name}')
        plt.legend(loc='lower right')
        plt.grid(True, alpha=0.3)
        
        # Ajouter quelques points de seuil sur la courbe ROC
        # Vérifier que thresholds n'est pas vide
        if len(thresholds) > 0:
            # Sélectionner des seuils plus pertinents
            threshold_points = [0.1, 0.3, 0.5, 0.7, 0.9]
            threshold_indices = []
            
            for t in threshold_points:
                # Vérifier que les seuils sont dans la plage des valeurs de thresholds
                if t >= min(thresholds) and t <= max(thresholds):
                    # Trouver l'indice du seuil le plus proche
                    idx = np.argmin(np.abs(thresholds - t))
                    threshold_indices.append(idx)
                
            # Marquer les points de seuil sur la courbe s'il y en a
            if threshold_indices:
                plt.scatter(fpr[threshold_indices], tpr[threshold_indices], c='red', s=50)
                
                # Ajouter les annotations pour les seuils
                for i, idx in enumerate(threshold_indices):
                    plt.annotate(f'Seuil ≈ {thresholds[idx]:.2f}', 
                                (fpr[idx], tpr[idx]), 
                                xytext=(10, -10), 
                                textcoords='offset points',
                                fontsize=8,
                                arrowprops=dict(arrowstyle='->', color='red', lw=1))
        
        plt.tight_layout()
        plt.show()
        
        # Visualiser la distribution des probabilités prédites
        plt.figure(figsize=(10, 6))
        
        # Séparer les prédictions par classe réelle
        normal_probs = y_pred_proba[y_test == 0]
        pneumonia_probs = y_pred_proba[y_test == 1]
        
        # Vérifier qu'il y a des exemples dans chaque classe
        if len(normal_probs) > 0 and len(pneumonia_probs) > 0:
            # Tracer les histogrammes
            plt.hist(normal_probs, bins=20, alpha=0.5, color='blue', label=class_names[0])
            plt.hist(pneumonia_probs, bins=20, alpha=0.5, color='red', label=class_names[1])
            
            plt.axvline(x=0.5, color='black', linestyle='--', label='Seuil (0.5)')
            plt.xlabel('Probabilité prédite de pneumonie')
            plt.ylabel('Nombre d\'images')
            plt.title(f'Distribution des probabilités prédites - {model_name}')
            plt.legend()
            plt.grid(True, alpha=0.3)
        else:
            plt.text(0.5, 0.5, "Pas assez d'exemples pour afficher la distribution", 
                    ha='center', va='center', fontsize=12)
            plt.title(f'Distribution des probabilités prédites - {model_name}')
        
        plt.tight_layout()
        plt.show()
        
        # Afficher des exemples de prédictions (correctes et incorrectes)
        plt.figure(figsize=(15, 10))
        
        # Trouver des exemples corrects et incorrects
        correct_indices = np.where(y_pred == y_test)[0]
        incorrect_indices = np.where(y_pred != y_test)[0]
        
        # Limiter le nombre d'exemples à afficher
        num_examples = min(3, len(correct_indices), len(incorrect_indices))
        
        if num_examples > 0:
            # Sélectionner des exemples aléatoires
            correct_samples = np.random.choice(correct_indices, num_examples, replace=False)
            incorrect_samples = np.random.choice(incorrect_indices, num_examples, replace=False)
            
            # Afficher les exemples corrects
            for i, idx in enumerate(correct_samples):
                plt.subplot(2, num_examples, i + 1)
                plt.imshow(X_test[idx])
                true_label = class_names[y_test[idx]]
                pred_label = class_names[y_pred[idx]]
                plt.title(f"Correct\nVrai: {true_label}\nPréd: {pred_label}\nConf: {y_pred_proba[idx]:.2f}")
                plt.axis('off')
            
            # Afficher les exemples incorrects
            for i, idx in enumerate(incorrect_samples):
                plt.subplot(2, num_examples, num_examples + i + 1)
                plt.imshow(X_test[idx])
                true_label = class_names[y_test[idx]]
                pred_label = class_names[y_pred[idx]]
                plt.title(f"Incorrect\nVrai: {true_label}\nPréd: {pred_label}\nConf: {y_pred_proba[idx]:.2f}")
                plt.axis('off')
            
            plt.suptitle(f"Exemples de prédictions - {model_name}", fontsize=16)
            plt.tight_layout()
            plt.subplots_adjust(top=0.9)
            plt.show()
        else:
            print("Pas assez d'exemples pour afficher des prédictions.")
        
        return metrics
    
    except Exception as e:
        print(f"Erreur lors de l'évaluation du modèle {model_name}: {e}")
        import traceback
        traceback.print_exc()
        return None

# Évaluer chaque modèle (si disponible)
models = {}

# Vérifier si les modèles existent avant de les ajouter au dictionnaire
# Ces variables doivent être définies avant d'exécuter ce code
try:
    if 'cnn_model' in globals() and cnn_model is not None:
        models['CNN'] = cnn_model
except NameError:
    print("Le modèle CNN n'est pas défini.")

try:
    if 'vgg16_model' in globals() and vgg16_model is not None:
        models['VGG16'] = vgg16_model
except NameError:
    print("Le modèle VGG16 n'est pas défini.")

try:
    if 'resnet50_model' in globals() and resnet50_model is not None:
        models['ResNet50'] = resnet50_model
except NameError:
    print("Le modèle ResNet50 n'est pas défini.")

# Vérifier si les données de test sont disponibles
if 'X_test' not in globals() or 'y_test' not in globals() or len(X_test) == 0:
    print("Les données de test ne sont pas disponibles. Impossible d'évaluer les modèles.")
else:
    # Si aucun modèle n'est disponible, afficher un message
    if not models:
        print("Aucun modèle n'est disponible pour l'évaluation. Vous devez d'abord entraîner les modèles.")
    else:
        # Libérer la mémoire avant l'évaluation
        import gc
        gc.collect()
        tf.keras.backend.clear_session()
        
        results = {}
        for model_name, model in models.items():
            print(f"\nÉvaluation du modèle {model_name}...")
            result = evaluate_model(model, X_test, y_test, model_name)
            if result is not None:
                results[model_name] = result
            
            # Libérer la mémoire après chaque évaluation
            gc.collect()
            tf.keras.backend.clear_session()

        # Créer un DataFrame pour comparer les métriques si des résultats sont disponibles
        if results:
            metrics_df = pd.DataFrame(results).T
            metrics_df = metrics_df[['accuracy', 'precision', 'recall', 'f1_score', 'auc']]

            # Afficher le tableau de comparaison
            print("\nComparaison des métriques:")
            print("=" * 50)
            print(metrics_df)

            # Visualiser les métriques sous forme de graphique à barres
            plt.figure(figsize=(12, 8))
            ax = metrics_df.plot(kind='bar', figsize=(12, 8))
            plt.title('Comparaison des métriques par modèle', fontsize=14)
            plt.xlabel('Modèle', fontsize=12)
            plt.ylabel('Score', fontsize=12)
            plt.ylim(0, 1)
            plt.legend(loc='lower right', fontsize=10)
            plt.grid(axis='y', alpha=0.3)
            
            # Ajouter les valeurs sur les barres
            for container in ax.containers:
                ax.bar_label(container, fmt='%.3f', fontsize=8)
                
            plt.tight_layout()
            plt.show()
            
            # Sauvegarder les résultats dans un fichier CSV
            try:
                results_dir = r'C:\Users\Krikri\Documents\EPITECH\MSc\Msc1\T-DEV-810-PAR-29\epitech_docs\detection_pneumonie_ia\results'
                os.makedirs(results_dir, exist_ok=True)
                metrics_df.to_csv(os.path.join(results_dir, 'model_comparison.csv'))
                print(f"Résultats sauvegardés dans {os.path.join(results_dir, 'model_comparison.csv')}")
            except Exception as e:
                print(f"Erreur lors de la sauvegarde des résultats: {e}")

## 6. Visualisation des prédictions

Nous allons visualiser quelques exemples de prédictions pour mieux comprendre les performances des modèles.

In [ ]:
def visualize_predictions(model, X_test, y_test, model_name='Modèle', num_samples=5):
    """
    Visualise quelques prédictions du modèle.
    
    Args:
        model: Modèle à utiliser pour les prédictions
        X_test (numpy.ndarray): Images de test
        y_test (numpy.ndarray): Étiquettes de test
        model_name (str): Nom du modèle
        num_samples (int): Nombre d'exemples à visualiser
    """
    # Vérifier si le modèle et les données sont disponibles
    if model is None:
        print(f"Le modèle {model_name} n'est pas disponible.")
        return
    
    if len(X_test) == 0 or len(y_test) == 0:
        print(f"Les données de test sont vides. Impossible de visualiser les prédictions du modèle {model_name}.")
        return
    
    try:
        # Limiter le nombre d'échantillons au nombre d'exemples disponibles
        num_samples = min(num_samples, len(X_test))
        
        # Faire des prédictions avec un batch_size adapté au modèle
        batch_size = 16
        if 'resnet' in model_name.lower() or 'vgg' in model_name.lower():
            batch_size = 8
        
        print(f"Prédiction sur {num_samples} images de test...")
        y_pred_proba = model.predict(X_test, batch_size=batch_size, verbose=0).flatten()
        y_pred = (y_pred_proba >= 0.5).astype(int)
        
        # Sélectionner différents types d'exemples
        # 1. Exemples correctement classifiés (vrais positifs et vrais négatifs)
        correct_indices = np.where(y_pred == y_test)[0]
        # 2. Exemples incorrectement classifiés (faux positifs et faux négatifs)
        incorrect_indices = np.where(y_pred != y_test)[0]
        
        # Vérifier s'il y a des exemples incorrects
        has_incorrect = len(incorrect_indices) > 0
        
        # Déterminer le nombre d'exemples à afficher de chaque type
        num_incorrect = min(num_samples // 2, len(incorrect_indices)) if has_incorrect else 0
        num_correct = num_samples - num_incorrect
        
        # Sélectionner les indices
        selected_indices = []
        
        if num_correct > 0 and len(correct_indices) > 0:
            selected_indices.extend(np.random.choice(correct_indices, num_correct, replace=False))
        
        if num_incorrect > 0:
            selected_indices.extend(np.random.choice(incorrect_indices, num_incorrect, replace=False))
        
        # Si nous n'avons pas assez d'exemples, compléter avec des exemples aléatoires
        if len(selected_indices) < num_samples:
            remaining = num_samples - len(selected_indices)
            all_indices = np.arange(len(X_test))
            remaining_indices = np.setdiff1d(all_indices, selected_indices)
            if len(remaining_indices) > 0:
                selected_indices.extend(np.random.choice(remaining_indices, min(remaining, len(remaining_indices)), replace=False))
        
        # Mélanger les indices pour ne pas avoir tous les exemples corrects puis tous les incorrects
        np.random.shuffle(selected_indices)
        
        # Visualiser les exemples
        plt.figure(figsize=(15, 3 * min(num_samples, len(selected_indices))))
        
        for i, idx in enumerate(selected_indices):
            plt.subplot(min(num_samples, len(selected_indices)), 1, i+1)
            
            # Afficher l'image
            plt.imshow(X_test[idx])
            
            # Déterminer la couleur en fonction de la prédiction
            is_correct = y_pred[idx] == y_test[idx]
            color = 'green' if is_correct else 'red'
            
            # Déterminer les libellés des classes
            true_label = 'Pneumonie' if y_test[idx] == 1 else 'Normal'
            pred_label = 'Pneumonie' if y_pred[idx] == 1 else 'Normal'
            
            # Ajouter le type d'erreur si la prédiction est incorrecte
            error_type = ""
            if not is_correct:
                if y_test[idx] == 0 and y_pred[idx] == 1:
                    error_type = " (Faux Positif)"
                else:
                    error_type = " (Faux Négatif)"
            
            # Afficher le titre avec les informations de prédiction
            plt.title(f"Vrai: {true_label}, Prédit: {pred_label}{error_type} (Confiance: {y_pred_proba[idx]:.2f})", 
                     color=color, fontsize=10)
            
            # Désactiver les axes
            plt.axis('off')
        
        plt.suptitle(f'Prédictions du modèle {model_name}', fontsize=16)
        plt.tight_layout(rect=[0, 0, 1, 0.97])
        plt.show()
        
        # Afficher un résumé des performances
        correct = np.sum(y_pred == y_test)
        total = len(y_test)
        accuracy = correct / total
        
        print(f"Résumé des performances du modèle {model_name}:")
        print(f"Accuracy: {accuracy:.4f} ({correct}/{total} prédictions correctes)")
        
        # Calculer les métriques par classe
        tp = np.sum((y_test == 1) & (y_pred == 1))  # Vrais positifs
        tn = np.sum((y_test == 0) & (y_pred == 0))  # Vrais négatifs
        fp = np.sum((y_test == 0) & (y_pred == 1))  # Faux positifs
        fn = np.sum((y_test == 1) & (y_pred == 0))  # Faux négatifs
        
        print(f"Classe 'Normal': {tn}/{tn+fp} prédictions correctes ({tn/(tn+fp)*100:.1f}%)")
        print(f"Classe 'Pneumonie': {tp}/{tp+fn} prédictions correctes ({tp/(tp+fn)*100:.1f}%)")
        
    except Exception as e:
        print(f"Erreur lors de la visualisation des prédictions du modèle {model_name}: {e}")
        import traceback
        traceback.print_exc()

# Visualiser les prédictions pour chaque modèle
if 'models' in globals() and models:
    # Libérer la mémoire avant de commencer
    import gc
    gc.collect()
    tf.keras.backend.clear_session()
    
    for model_name, model in models.items():
        print(f"\nVisualisation des prédictions du modèle {model_name}:")
        visualize_predictions(model, X_test, y_test, model_name)
        
        # Libérer la mémoire après chaque modèle
        gc.collect()
        tf.keras.backend.clear_session()
else:
    print("Aucun modèle n'est disponible pour visualiser les prédictions.")

## 7. Conclusion

Dans ce notebook, nous avons entraîné différents modèles pour la détection de pneumonie à partir d'images radiographiques :
1. Un CNN simple
2. Un modèle utilisant le transfer learning avec VGG16
3. Un modèle utilisant le transfer learning avec ResNet50

Nous avons évalué leurs performances sur l'ensemble de test et visualisé les résultats. Les résultats montrent que [à compléter avec vos observations].

Les modèles entraînés ont été sauvegardés dans le répertoire `../results` et peuvent être utilisés pour faire des prédictions sur de nouvelles images.

## 7. Conclusion

Dans ce notebook, nous avons entraîné différents modèles pour la détection de pneumonie à partir d'images radiographiques :
1. Un CNN simple
2. Un modèle utilisant le transfer learning avec VGG16
3. Un modèle utilisant le transfer learning avec ResNet50

Nous avons évalué leurs performances sur l'ensemble de test et visualisé les résultats. Les résultats montrent que [à compléter avec vos observations].

Les modèles entraînés ont été sauvegardés dans le répertoire `../results` et peuvent être utilisés pour faire des prédictions sur de nouvelles images.